In [24]:
from extract_zip import extract_zip_to_memory # function for decompressing zip in memory
from PIL import Image, ImageOps # image handling
import pandas as pd # data manipulation
import numpy as np

import keras
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential

from sklearn import preprocessing

basepath = "/mnt/datasets/plankton/flowcam/"

In [2]:
img_files = extract_zip_to_memory(basepath + "imgs.zip")

In [3]:
img_files_keys = img_files.keys()
print(len(img_files_keys))
list(img_files_keys)[:5]

243610


['imgs/32536872.jpg',
 'imgs/32562298.jpg',
 'imgs/32576701.jpg',
 'imgs/32664455.jpg',
 'imgs/32708747.jpg']

In [4]:
# dataset
features_native_RawDF = pd.read_csv(basepath + 'features_native.csv.gz')
meta_RawDF = pd.read_csv(basepath + 'meta.csv')
taxo_RawDF = pd.read_csv(basepath + 'taxo.csv')

In [5]:
print(features_native_RawDF.shape)
print(meta_RawDF.shape)
print(taxo_RawDF.shape)

(243610, 65)
(243610, 14)
(178, 8)


In [6]:
# change datatype of meta
meta_RawDF['objid'] = meta_RawDF['objid'].astype(np.int64, errors='ignore')

In [7]:
max_height = features_native_RawDF.height.max()
max_width = features_native_RawDF.width.max()
print(max_height)
print(max_width)

739
972


In [8]:
# code inspired by https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/

def image_processing(path):
    desired_size = 972 # max of width and height in dataset
    im_orginal = Image.open(path)
    old_size = im_orginal.size  # old_size[0] is in (width, height) format
    
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im_scaled = im_orginal.resize(new_size) # scale up picture before padding to keep information
    
    delta_w = desired_size - new_size[0]
    delta_h = desired_size - new_size[1]
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    im_padded = ImageOps.expand(im_scaled, padding, fill=255) # padding of scaled picture
    
    final_size = (100, 100) # input size for CNN
    im_final = im_padded.resize(final_size, resample=0) # resize to input size of CNN
    return np.array(im_final)
    
test = image_processing(img_files['imgs/32582800.jpg']) # test
print(test.shape)


(100, 100)


In [20]:
# processes image before training

def autoloader(list_of_keys):
    return np.array([image_processing(img_files[key]) for key in list_of_keys])
    

In [37]:
# input dimensions
input_shape = (100, 100, 1)

# network parameters 
batch_size = 128
classes = meta_RawDF.level2.unique()
num_classes = meta_RawDF.level2.nunique()
epochs = 5 # Further Fine Tuning can be done


In [41]:
# split to train and test data

# 9000 first elements as train
train_keys = list(img_files_keys)[:900]
X_train = autoloader(train_keys)
y_train = pd.DataFrame(columns=['label1', 'label2'])

for key in train_keys:
    key_id = int(key[5:13])
    label1 = meta_RawDF.loc[meta_RawDF.objid == key_id].level1.values[0]
    label2 = meta_RawDF.loc[meta_RawDF.objid == key_id].level2.values[0]
    y_train.loc[key_id] = [label1, label2]
    


# 1000 next elements as test
test_keys = list(img_files_keys)[900:1000]
X_test = autoloader(test_keys)
y_test = pd.DataFrame(columns=['label1', 'label2'])

for key in test_keys:
    key_id = int(key[5:13])
    label1 = meta_RawDF.loc[meta_RawDF.objid == key_id].level1.values[0]
    label2 = meta_RawDF.loc[meta_RawDF.objid == key_id].level2.values[0]
    y_test.loc[key_id] = [label1, label2]


# drop label 1 or label 2
y_train = y_train.drop(['label1'], axis=1)
y_test = y_test.drop(['label1'], axis=1)

# convert the target variable  
le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# reshaping
X_train = X_train.reshape(X_train.shape[0], 100, 100, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 1)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: unorderable types: float() < str()

In [39]:
# Keras CNN model

model = Sequential()

# add first convolutional layer
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# add second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# add one max pooling layer 
model.add(MaxPooling2D(pool_size=(2, 2)))

# add one dropout layer
model.add(Dropout(0.25))

# add flatten layer
model.add(Flatten())

# add dense layer
model.add(Dense(128, activation='relu'))

# add another dropout layer
model.add(Dropout(0.5))

# add dense layer
model.add(Dense(num_classes, activation='softmax'))

# complile the model and view its architecur
model.compile(loss=keras.losses.categorical_crossentropy,  optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 147456)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               18874496  
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [40]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
accuracy = model.evaluate(x_val, y_val, verbose=0)
print('Test accuracy:', accuracy[1])

ValueError: Error when checking input: expected conv2d_3_input to have 4 dimensions, but got array with shape (10, 100, 100)